## FEATURE ENGINEERING 1
- Create dataframe series which specifies if location and keywords are missing
- Create a model with this columns (depending on the metrics generated)
- Combine with text model

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


### CREATE DATAFRAME SERIES (empty_location and empty_keyword)

In [16]:
train["keyword"] = train["keyword"].fillna("None")
train["location"] = train["location"].fillna("None")

In [17]:
def empty_keyword(df):
    if df == "None":
        return 0
    else:
        return 1

In [18]:
train["empty_keyword"] = train["keyword"].apply(empty_keyword)
train["empty_location"] = train["location"].apply(empty_keyword)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
id                7613 non-null int64
keyword           7613 non-null object
location          7613 non-null object
text              7613 non-null object
target            7613 non-null int64
empty_keyword     7613 non-null int64
empty_location    7613 non-null int64
dtypes: int64(4), object(3)
memory usage: 416.5+ KB


In [59]:
train["keyword"].value_counts()

None                     61
fatalities               45
armageddon               42
deluge                   42
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 222, dtype: int64

In [20]:
train["empty_keyword"].value_counts()

1    7552
0      61
Name: empty_keyword, dtype: int64

In [21]:
train["empty_location"].value_counts()

1    5080
0    2533
Name: empty_location, dtype: int64

In [22]:
train["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [23]:
feature_cols = ["empty_keyword", "empty_location"]

In [24]:
X = train[feature_cols]
X.head()

,empty_keyword,empty_location
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [54]:
X_eins = train["empty_location"]
X_eins = X_eins.values.reshape(-1, 1)

In [25]:
y = train["target"]

### CREATE A MODEL WITH empty_location and empty_keyword

In [52]:
from sklearn.model_selection import cross_val_score

def run_model(model):
    return cross_val_score(model, X, y, cv = 5, scoring = "accuracy").mean()

def run_model_eins(model):
    return cross_val_score(model, X_eins, y, cv = 5, scoring = "accuracy").mean()

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [55]:
print(run_model(lr))
print(run_model_eins(lr))

0.5702089760861372
0.5703402096819377


In [56]:
print(run_model(nb))
print(run_model_eins(nb))

0.5703402096819377
0.5703402096819377


In [57]:
print(run_model(rf))
print(run_model_eins(rf))

0.5702089760861372
0.5703402096819377


In [58]:
print(run_model(xgb))
print(run_model_eins(xgb))

0.5702089760861372
0.5703402096819377


#### Conclusion: the scores are not much different from the NULL-MODEL 